In [ ]:
!nvidia-smi

Thu Mar 24 05:15:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lakshmiprasanna95","key":"bce96094ad02fac0221a2472fd3b4188"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d fereshtej/cq500-yolov5-model2

 99% 1.50G/1.51G [00:26<00:00, 74.2MB/s]
100% 1.51G/1.51G [00:26<00:00, 61.6MB/s]


In [ ]:
! unzip cq500-yolov5-model2

Streaming output truncated to the last 5000 lines.
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714634.2090176_val_970.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714634.2090202_val_1475.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714634.2090208_val_1784.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714634.2090244_val_4157.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714637.2090671_val_2175.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714637.2090761_val_1483.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714637.2090769_val_4205.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714637.2090775_val_1041.png  
  inflating: CQ500_Dataset/val/images/1.2.276.0.7230010.3.1.4.296485376.1.1521714637.2

In [ ]:
!pip install -r /content/yolov5/requirements.txt

     |████████████████████████████████| 596 kB 14.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
with open('new_train_yaml', 'w+') as file:
    file.write(
        """
      # Parameters
nc: 80  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3TR, [1024]],  # 9 <--- C3TR() Transformer module
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
        """
    )


In [ ]:
with open('new_data_yaml', 'w+') as file:
    file.write(
        """
        train: /content/CQ500_Dataset/train
        val: /content/CQ500_Dataset/val

        nc: 5
        names: ['Epidural', 'Intraparenchymal','Intraventricular', 'Subarachnoid', 'Subdural']
        """
    )


In [ ]:
!python /content/yolov5/train.py --img 256 --batch 16 --epochs 50 --data /content/new_data_yaml --cfg /content/new_train_yaml --weights yolov5x.pt

train: weights=yolov5x.pt, cfg=/content/new_train_yaml, data=/content/new_data_yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=50, batch_size=16, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 1388, done.
remote: Counting objects: 100% (415/415), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 1388 (delta 413), reused 407 (delta 407), pack-reused 973
Receiving objects: 100% (1388/1388), 743.96 KiB | 4.51 MiB/s, done.
Resolving deltas: 100% (971/971), completed with 71 local objects.
From https://github.com/ultralytics/yolov5
   

In [ ]:
!python /content/yolov5/val.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --data /content/new_data_yaml

val: data=/content/new_data_yaml, weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-172-gfb83929 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 450 layers, 84977290 parameters, 0 gradients, 197.8 GFLOPs
val: Scanning '/content/CQ500_Dataset/val/labels.cache' images and labels... 4227 found, 0 missing, 0 empty, 0 corrupted: 100% 4227/4227 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 133/133 [05:00<00:00,  2.26s/it]
                 all       4227       6726      0.685      0.606      0.633      0.285
            Epidural       4227        104      0.915      0.875      0.928      0.529
    I

In [ ]:
!python /content/yolov5/val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/new_data_yaml

val: data=/content/new_data_yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-172-gfb83929 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 444 layers, 86200330 parameters, 0 gradients, 204.1 GFLOPs
val: Scanning '/content/CQ500_Dataset/val/labels.cache' images and labels... 4227 found, 0 missing, 0 empty, 0 corrupted: 100% 4227/4227 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 133/133 [04:45<00:00,  2.15s/it]
                 all       4227       6726      0.678      0.557        0.6      0.265
            Epidural       4227        104      0.925      0.837      0.906      0.509
    In

In [ ]:
!python /content/yolov5/val.py --weights /content/yolov5/runs/train/exp/weights/best.pt /content/yolov5/runs/train/exp2/weights/best.pt --data /content/new_data_yaml

val: data=/content/new_data_yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt', '/content/yolov5/runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-172-gfb83929 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 444 layers, 86200330 parameters, 0 gradients, 204.1 GFLOPs
Fusing layers... 
Model Summary: 450 layers, 84977290 parameters, 0 gradients, 197.8 GFLOPs
Ensemble created with ['/content/yolov5/runs/train/exp/weights/best.pt', '/content/yolov5/runs/train/exp2/weights/best.pt']

val: Scanning '/content/CQ500_Dataset/val/labels.cache' images and labels... 4227 found, 0 missing, 0 empty, 0 corrupted: 100% 4227/4227 [00:00<?, ?it/s]
               Class     Images     La

In [ ]:
!python /content/yolov5/val.py --weights /content/yolov5/runs/train/exp/weights/best.pt /content/drive/MyDrive/weights_l/best.pt --data /content/new_data_yaml

val: data=/content/new_data_yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt', '/content/drive/MyDrive/weights_l/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-172-gfb83929 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 444 layers, 86200330 parameters, 0 gradients, 204.1 GFLOPs
Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients, 107.9 GFLOPs
Ensemble created with ['/content/yolov5/runs/train/exp/weights/best.pt', '/content/drive/MyDrive/weights_l/best.pt']

val: Scanning '/content/CQ500_Dataset/val/labels.cache' images and labels... 4227 found, 0 missing, 0 empty, 0 corrupted: 100% 4227/4227 [00:00<?, ?it/s]
               Class     Images     Labels          

In [ ]:
!python /content/yolov5/val.py --weights /content/yolov5/runs/train/exp2/weights/best.pt /content/drive/MyDrive/weights_l/best.pt --data /content/new_data_yaml

val: data=/content/new_data_yaml, weights=['/content/yolov5/runs/train/exp2/weights/best.pt', '/content/drive/MyDrive/weights_l/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-172-gfb83929 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 450 layers, 84977290 parameters, 0 gradients, 197.8 GFLOPs
Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients, 107.9 GFLOPs
Ensemble created with ['/content/yolov5/runs/train/exp2/weights/best.pt', '/content/drive/MyDrive/weights_l/best.pt']

val: Scanning '/content/CQ500_Dataset/val/labels.cache' images and labels... 4227 found, 0 missing, 0 empty, 0 corrupted: 100% 4227/4227 [00:00<?, ?it/s]
               Class     Images     Labels        

In [ ]:
!python /content/yolov5/val.py --weights /content/drive/MyDrive/weights_l/best.pt --data /content/new_data_yaml

val: data=/content/new_data_yaml, weights=['/content/drive/MyDrive/weights_l/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-172-gfb83929 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients, 107.9 GFLOPs
val: Scanning '/content/CQ500_Dataset/val/labels.cache' images and labels... 4227 found, 0 missing, 0 empty, 0 corrupted: 100% 4227/4227 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 133/133 [02:46<00:00,  1.26s/it]
                 all       4227       6726      0.651      0.572      0.602      0.262
            Epidural       4227        104      0.851      0.933      0.938      0.507
    Intrapar